### ***Importando as bibliotecas***

In [1]:
import torch
import gc
import random
from datasets import load_dataset
from unsloth import FastLanguageModel
from transformers import TextStreamer

Unsloth: Your Flash Attention 2 installation seems to be broken?
A possible explanation is you have a new CUDA version which isn't
yet compatible with FA2? Please file a ticket to Unsloth or FA2.
We shall now use Xformers instead, which does not have any performance hits!
We found this negligible impact by benchmarking on 1x A100.
🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


### ***Inicializando o Modelo***

**Prompt**

Para estruturar os dados de validação vai ser utilizado o Alpaca, com isso vai ser possível fornecer ao modelo uma estrutura consistente que vai incluir:
- **Instrução**
- **Input (Contexto adicional)**

In [2]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

**Função de inicialização**

Recebe como parâmetros o modelo e o input e realiza os seguintes passos:

**1.** Limpeza de memória.
- É realizado a limpeza na memória para o modelo que será carregado.

**2.** Inicialização do modelo.
- Inícia o modelo e o tokenizer que foi passado por parâmetro.
- Habilita o modo de inferência

**3.** Criação do input e do streamer.
- Realiza a formatação do *alpaca_prompt* utilizando os parâmetros de entrada.

**4.** Gerar o texto
- Realiza a geração do novo texto.

In [3]:
# Inicia o modelo e realiza a geração de texto
def iniciar_modelo(model_name, input):
    # Limpar a memória da GPU
    torch.cuda.empty_cache()
    gc.collect()
    
    max_seq_length = 256 # Choose any! We auto support RoPE Scaling internally!
    dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
    load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

    # Inicializa o modelo e o tokenizer
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = model_name,
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
    )
    
    # Habilita o modo de inferência rápida
    FastLanguageModel.for_inference(model) # Enable native 2x faster inference
    
    # Instrução para o modelo
    instruction = 'Generate a detailed product description based on the following title:'
    
    # Cria o input para o modelo
    inputs = tokenizer(
        [
            alpaca_prompt.format(instruction, input, "")
        ], return_tensors = "pt").to("cuda")

    # Cria o streamer de texto
    text_streamer = TextStreamer(tokenizer)
    
    # Gera o texto
    _ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

### **Carregando o dataset**

Carrega o dataset que possui apenas o campo 'title' preenchido.

In [4]:
# Caminho do arquivo JSON do dataset de validação
caminho_arquivo = './LF-Amazon-1.3M/avaliacao/lbl.json'

# Carregar o dataset diretamente do arquivo JSON
dataset_avaliacao = load_dataset('json', data_files=caminho_arquivo, split='train')
print(dataset_avaliacao[:5])

{'uid': ['0000032050', 'B00D0DJAEG', 'B00D0F450I', 'B00D2JTMS2', 'B00D0FDUAY'], 'title': ['Adult Ballet Tutu Purple', 'Adult Ballet Tutu Pastel Rainbow', 'Adult Ballet Tutu Black, one size fit most', 'Adult Tutu Assorted Colors (Hot Pink)', 'Adult Ballet Tutu Red'], 'content': ['', '', '', '', ''], 'target_ind': [[], [], [], [], []], 'target_rel': [[], [], [], [], []]}


Função para retornar um exemplo aleatório do dataset.

In [5]:
def recupera_exemplo_aleatorio(dataset_avaliacao):
    # Escolher um exemplo aleatório do dataset
    exemplo = random.choice(dataset_avaliacao)

    title = exemplo['title']

    return title

### **Validação**

A validação será realizada recuperando um input aleatório do dataset, iniciando e gerando um texto com o Foundation Model e o Modelo onde foi realizado o Fine Tuning.

In [19]:
# Recupera um exemplo aleatório do dataset
input = recupera_exemplo_aleatorio(dataset_avaliacao)
print(input)

Rhonda Allison Amino Peptide Moisturizer 1.7oz


In [20]:
# Tiny Llama
print("--- Tiny Llama ---")
iniciar_modelo("unsloth/tinyllama-bnb-4bit", input)

--- Tiny Llama ---
==((====))==  Unsloth 2024.8: Fast Llama patching. Transformers = 4.43.3.
   \\   /|    GPU: NVIDIA GeForce RTX 4070 Ti. Max memory: 11.994 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.3.0+cu121. CUDA = 8.9. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.26.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
<s> Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Generate a detailed product description based on the following title:

### Input:
Rhonda Allison Amino Peptide Moisturizer 1.7oz

### Response:

Rhonda Allison Amino Peptide Moisturizer 1.7oz

### Instruction:
Generate a detailed product description based on the following title:

### Input:
Rhonda Allison Amino Peptide Moisturizer 1.7oz

### Response:

Rhonda Allison Amino Peptide Moisturizer 1.7oz

### Instruction:
Genera

In [21]:
# Modelo com o Fine Tuning
print("--- Fine Tuning ---")
iniciar_modelo("./model/lora_model_final", input)

--- Fine Tuning ---
==((====))==  Unsloth 2024.8: Fast Llama patching. Transformers = 4.43.3.
   \\   /|    GPU: NVIDIA GeForce RTX 4070 Ti. Max memory: 11.994 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.3.0+cu121. CUDA = 8.9. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.26.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
<s> Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Generate a detailed product description based on the following title:

### Input:
Rhonda Allison Amino Peptide Moisturizer 1.7oz

### Response:
Amino Peptide Moisturizer is a lightweight, non-greasy, non-comedogenic moisturizer that helps to hydrate and protect the skin. It is formulated with a blend of amino acids, peptides, and vitamins to help restore the skin's natural moisture balance.</s>
